In [1]:
import paddle
from paddle.nn import Linear
import paddle.nn.functional as F
import numpy as np
import os
import random

In [9]:
def load_data():
    datafile = './dataset/housing.data'
    data = np.fromfile(datafile, sep=' ', dtype=np.float32)
    
    feature_names = [ 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', \
                      'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV' ]
    feature_nums = len(feature_names)
    
    data = data.reshape([data.shape[0] // feature_nums, feature_nums])
    
    ratio = 0.8
    offset = int(data.shape[0] * ratio)
    training_data = data[:offset]
    
    maximums, minimums, avgs = training_data.max(axis=0), training_data.min(axis=0), \
                                training_data.sum(axis=0) / training_data.shape[0]
    
    global max_values
    global min_values
    global avg_values
    max_values = maximums
    min_values = minimums
    avg_values = avgs
    
    for i in range(feature_nums):
        data[:, i] = (data[:, i] - avgs[i]) / (maximums[i] - minimums[i])
    
    training_data = data[:offset]
    test_data = data[offset:]
    
    return training_data, test_data

In [10]:
class Regressor(paddle.nn.Layer):
    def __init__(self, 
                num_of_weights):
        
        super(Regressor, self).__init__()
        
        self.num_of_weights = num_of_weights
        # 定义一层全连接层，输入维度是13，输出维度是1
        self.fc = Linear(in_features = self.num_of_weights, out_features=1)
    
    def forward(self, inputs):
        z = self.fc(inputs)
        
        return z

In [11]:
model = Regressor(13)
# 开启模型的训练模式
model.train()

training_data, test_data = load_data()

'''定义优化算法，使用随机梯度下降SGD'''
opt = paddle.optimizer.SGD(learning_rate=1e-2, parameters=model.parameters())

In [12]:
num_of_epochs = 10
batch_size = 10

for epoch_id in range(num_of_epochs):
    N = len(training_data)
    np.random.shuffle(training_data)
    mini_batches = [training_data[k:k+batch_size] for k in range(0, N, batch_size)]
    
    for iter_id, mini_batch in enumerate(mini_batches):
        x = np.array(training_data[:, :-1])
        y = np.array(training_data[:, -1:])
        housing_features = paddle.to_tensor(x)
        prices = paddle.to_tensor(y)
        
        # 前向计算
        predicts = model(housing_features)
        # 计算损失
        loss = F.square_error_cost(predicts, label=prices)
        avg_loss = paddle.mean(loss)
        if iter_id % 20 == 0:
            print('epoch_id:{}, iter_id:{}, loss:{}'.format(epoch_id, iter_id, avg_loss.numpy()))
        
        # 反向传播
        avg_loss.backward()
        # 最小化loss，更新参数
        opt.step()
        # 清除梯度
        opt.clear_grad()

epoch_id:0, iter_id:0, loss:[0.05239948]
epoch_id:0, iter_id:20, loss:[0.05044193]
epoch_id:0, iter_id:40, loss:[0.04870461]
epoch_id:1, iter_id:0, loss:[0.04862281]
epoch_id:1, iter_id:20, loss:[0.04707587]
epoch_id:1, iter_id:40, loss:[0.04567946]
epoch_id:2, iter_id:0, loss:[0.04561313]
epoch_id:2, iter_id:20, loss:[0.04434928]
epoch_id:2, iter_id:40, loss:[0.04319226]
epoch_id:3, iter_id:0, loss:[0.04313693]
epoch_id:3, iter_id:20, loss:[0.04207582]
epoch_id:3, iter_id:40, loss:[0.04109351]
epoch_id:4, iter_id:0, loss:[0.04104628]
epoch_id:4, iter_id:20, loss:[0.04013606]
epoch_id:4, iter_id:40, loss:[0.03928618]
epoch_id:5, iter_id:0, loss:[0.03924514]
epoch_id:5, iter_id:20, loss:[0.03845138]
epoch_id:5, iter_id:40, loss:[0.03770537]
epoch_id:6, iter_id:0, loss:[0.03766922]
epoch_id:6, iter_id:20, loss:[0.03696828]
epoch_id:6, iter_id:40, loss:[0.03630618]
epoch_id:7, iter_id:0, loss:[0.03627402]
epoch_id:7, iter_id:20, loss:[0.03564903]
epoch_id:7, iter_id:40, loss:[0.03505635]


In [13]:
paddle.save(model.state_dict(), './model/LR_model.pdparams')
print('模型保存成功')

模型保存成功


In [14]:
'''测试模型'''
def load_one_example():
    idx = np.random.randint(0, test_data.shape[0])
    idx = -10
    one_data, label = test_data[idx, :-1], test_data[idx, -1]
    one_data = one_data.reshape([1, -1])
    
    return one_data, label

# 参数为保存模型参数的文件地址
model_dict = paddle.load('LR_model.pdparams')
model.load_dict(model_dict)
model.eval()

# 参数为数据集的文件地址
one_data, label = load_one_example()
# 将数据转为动态图的variable格式 
one_data = paddle.to_tensor(one_data)
predict = model(one_data)

# 对结果做反归一化处理
predict = predict * (max_values[-1] - min_values[-1]) + avg_values[-1]
# 对label数据做反归一化处理
label = label * (max_values[-1] - min_values[-1]) + avg_values[-1]

print("Inference result is {}, the corresponding label is {}".format(predict.numpy(), label))


Inference result is [[15.646884]], the corresponding label is 19.700000762939453


In [2]:
def load_data():
    # 从文件导入数据
    datafile = './dataset/housing.data'
    data = np.fromfile(datafile, sep=' ', dtype=np.float32)

    # 每条数据包括14项，其中前面13项是影响因素，第14项是相应的房屋价格中位数
    feature_names = [ 'CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', \
                      'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'MEDV' ]
    feature_num = len(feature_names)

    # 将原始数据进行Reshape，变成[N, 14]这样的形状
    data = data.reshape([data.shape[0] // feature_num, feature_num])

    # 将原数据集拆分成训练集和测试集
    # 这里使用80%的数据做训练，20%的数据做测试
    # 测试集和训练集必须是没有交集的
    ratio = 0.8
    offset = int(data.shape[0] * ratio)
    training_data = data[:offset]

    # 计算train数据集的最大值，最小值，平均值
    maximums, minimums, avgs = training_data.max(axis=0), training_data.min(axis=0), \
                                 training_data.sum(axis=0) / training_data.shape[0]
    
    # 记录数据的归一化参数，在预测时对数据做归一化
    global max_values
    global min_values
    global avg_values
    max_values = maximums
    min_values = minimums
    avg_values = avgs

    # 对数据进行归一化处理
    for i in range(feature_num):
        data[:, i] = (data[:, i] - avgs[i]) / (maximums[i] - minimums[i])

    # 训练集和测试集的划分比例
    training_data = data[:offset]
    test_data = data[offset:]
    return training_data, test_data

D:\Path\Anaconda\lib\site-packages\ipykernel\ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [3]:
class Regressor(paddle.nn.Layer):

    # self代表类的实例自身
    def __init__(self):
        # 初始化父类中的一些参数
        super(Regressor, self).__init__()
        
        # 定义一层全连接层，输入维度是13，输出维度是1
        self.fc = Linear(in_features=13, out_features=1)
    
    # 网络的前向计算
    def forward(self, inputs):
        x = self.fc(inputs)
        return x

In [4]:
# 声明定义好的线性回归模型
model = Regressor()
# 开启模型训练模式
model.train()
# 加载数据
training_data, test_data = load_data()
# 定义优化算法，使用随机梯度下降SGD
# 学习率设置为0.01
opt = paddle.optimizer.SGD(learning_rate=0.01, parameters=model.parameters())

In [5]:
EPOCH_NUM = 10   # 设置外层循环次数
BATCH_SIZE = 10  # 设置batch大小

# 定义外层循环
for epoch_id in range(EPOCH_NUM):
    # 在每轮迭代开始之前，将训练数据的顺序随机的打乱
    np.random.shuffle(training_data)
    # 将训练数据进行拆分，每个batch包含10条数据
    mini_batches = [training_data[k:k+BATCH_SIZE] for k in range(0, len(training_data), BATCH_SIZE)]
    # 定义内层循环
    for iter_id, mini_batch in enumerate(mini_batches):
        x = np.array(mini_batch[:, :-1]) # 获得当前批次训练数据
        y = np.array(mini_batch[:, -1:]) # 获得当前批次训练标签（真实房价）
        # 将numpy数据转为飞桨动态图tensor形式
        house_features = paddle.to_tensor(x)
        prices = paddle.to_tensor(y)
        
        # 前向计算
        predicts = model(house_features)
        
        # 计算损失
        loss = F.square_error_cost(predicts, label=prices)
        avg_loss = paddle.mean(loss)
        if iter_id%20==0:
            print("epoch: {}, iter: {}, loss is: {}".format(epoch_id, iter_id, avg_loss.numpy()))
        
        # 反向传播
        avg_loss.backward()
        # 最小化loss,更新参数
        opt.step()
        # 清除梯度
        opt.clear_grad()

epoch: 0, iter: 0, loss is: [0.10110427]
epoch: 0, iter: 20, loss is: [0.0863585]
epoch: 0, iter: 40, loss is: [0.18885991]
epoch: 1, iter: 0, loss is: [0.05490042]
epoch: 1, iter: 20, loss is: [0.16416094]
epoch: 1, iter: 40, loss is: [0.04386979]
epoch: 2, iter: 0, loss is: [0.09962933]
epoch: 2, iter: 20, loss is: [0.07871637]
epoch: 2, iter: 40, loss is: [0.00818549]
epoch: 3, iter: 0, loss is: [0.05975778]
epoch: 3, iter: 20, loss is: [0.08919506]
epoch: 3, iter: 40, loss is: [0.19367181]
epoch: 4, iter: 0, loss is: [0.0722723]
epoch: 4, iter: 20, loss is: [0.04011645]
epoch: 4, iter: 40, loss is: [0.08961165]
epoch: 5, iter: 0, loss is: [0.13970749]
epoch: 5, iter: 20, loss is: [0.06741773]
epoch: 5, iter: 40, loss is: [0.08289269]
epoch: 6, iter: 0, loss is: [0.05464761]
epoch: 6, iter: 20, loss is: [0.04843052]
epoch: 6, iter: 40, loss is: [0.06518123]
epoch: 7, iter: 0, loss is: [0.04641677]
epoch: 7, iter: 20, loss is: [0.07691812]
epoch: 7, iter: 40, loss is: [0.06149705]
ep

In [6]:
# 保存模型参数，文件名为LR_model.pdparams
paddle.save(model.state_dict(), 'LR_model.pdparams')
print("模型保存成功，模型参数保存在LR_model.pdparams中")

模型保存成功，模型参数保存在LR_model.pdparams中


In [7]:
def load_one_example():
    # 从上边已加载的测试集中，随机选择一条作为测试数据
    idx = np.random.randint(0, test_data.shape[0])
    idx = -10
    one_data, label = test_data[idx, :-1], test_data[idx, -1]
    # 修改该条数据shape为[1,13]
    one_data =  one_data.reshape([1,-1])

    return one_data, label

In [8]:
# 参数为保存模型参数的文件地址
model_dict = paddle.load('LR_model.pdparams')
model.load_dict(model_dict)
model.eval()

# 参数为数据集的文件地址
one_data, label = load_one_example()
# 将数据转为动态图的variable格式 
one_data = paddle.to_tensor(one_data)
predict = model(one_data)

# 对结果做反归一化处理
predict = predict * (max_values[-1] - min_values[-1]) + avg_values[-1]
# 对label数据做反归一化处理
label = label * (max_values[-1] - min_values[-1]) + avg_values[-1]

print("Inference result is {}, the corresponding label is {}".format(predict.numpy(), label))

Inference result is [[15.469454]], the corresponding label is 19.700000762939453


D:\Path\Anaconda\lib\site-packages\paddle\fluid\dygraph\math_op_patch.py:66: DeprecationWarning: an integer is required (got type numpy.float32).  Implicit conversion to integers using __int__ is deprecated, and may be removed in a future version of Python.
  out = core.ops.fill_constant(out, 'dtype', dtype, 'shape', shape,
